bullshit bro indeed

# Content（可直接 ctrl+F 相关索引）
## 抓取股票
> 1（我承认这里有define function）\
> 2（但这里没有自己define的）
## 双均线策略，以 20 天和 60 天为例
## MMD
## 最大回撤期
## 设置一个rolling期（以41天为例）以及一个上下波动的阈值（以4为例）
>1.单纯画图\
>2.画出买入卖出点\
>3.对比strategy return和原股票return（我认为大部分算return都可以按这个来）
## sharpe ratio
## Compound Annual Growth Rate，CAGR
## 抓取股票后由一个rolling 以及一个阈值（这里是两倍std）确定的上下限，制定策略
> 1.画图\
> 2.计算sharpe ratio
## 用之前的一定天数（以 3 天为例）预测明天的log return
## MACD策略，长期(26天)短期(12天)差值与MACD signal(9天)
## find the `five` worst drawdown periods over the investment horizon and provide their net drawdown in % and duration, respectively.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from pandas_datareader import data as pdr

import datetime
import yfinance as yf
import pandas_datareader.data as web 

(
    yf
    .pdr_override()
)
np.set_printoptions(precision = 3)

plt.style.use("ggplot")

mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.color"] = "grey"
mpl.rcParams["grid.alpha"] = 0.25

mpl.rcParams["axes.facecolor"] = "white"

mpl.rcParams["legend.fontsize"] = 14

## 抓取股票
### 1

In [ ]:
stocks =\
    ["WMT", # Note that FB has been changed to META
     "IBM"]
def extract_sp(stocks, start, end):
    def data(ticker):
        return(pdr.get_data_yahoo(ticker,
                                  start = start,
                                  end = end)
              )
    FAANG_Stock = map(data, stocks)
    return(pd.concat(FAANG_Stock,
                     keys = stocks,
                     names = ["Tickers", "Date"]
                    )
          )
#设置起始和结束时间
FAANG =\
    extract_sp(stocks,
               datetime.datetime(2014, 10, 13),
               datetime.datetime(2022, 9, 14)
              )
Daily_Closing_Price =\
(
FAANG[["Adj Close"]]
    .reset_index()
    .pivot(index = "Date",
           columns = "Tickers",
           values = "Adj Close")
)
Daily_Closing_Price

### 2（这里没有定义函数）

In [ ]:
google =\
(    web
     .get_data_yahoo("GOOGL", 
                     start, 
                     end)
)

## 双均线策略，以 20 天和 60 天为例
>注意只是画了红色的矩形图，

In [ ]:
FAANG2 =Daily_Closing_Price["IBM"].copy()
FAANG2 = pd.DataFrame(FAANG2)

#更改天数在这里
FAANG2["SMA_20"] =\
(
    FAANG2
    ["IBM"]
    .rolling(window = 20)
    .mean()
)
FAANG2["SMA_60"] =\
(
    FAANG2
    ["IBM"]
    .rolling(60)
    .mean()
)

#确定position
FAANG2["positions"] =\
(
    np
    .where(FAANG2["SMA_20"] > FAANG2["SMA_60"],
           1, -1)
)
FAANG2 =\
(
    FAANG2
    .dropna()
)

ax =\
(
    FAANG2
    [["IBM", 
      "SMA_20", 
      "SMA_60", 
      "positions"]]
    .plot(secondary_y = "positions",
          color = ["grey", 
                   "blue",
                   "green",
                   "red"],
          style = ["-",
                  "--",
                  "--",
                  "-"],
          figsize = [18, 6]
         )
)

(
    ax
    .legend(loc = "upper center",
            shadow = True,
            ncol = 4,
            bbox_to_anchor = (0.55, 1.10),
            fancybox = True
           )
)

## MMD

In [ ]:
FAANG2["log_returns"] =\
(
    np
    .log(FAANG2["Adj Close"] / FAANG2["Adj Close"].shift(1)
        )
)
FAANG2["strategy_returns"] =\
(
    FAANG2["positions"]
    .shift(1)
    *
    FAANG2["log_returns"] # passive following
)
FAANG2["cumulative_returns"] =\
(
    FAANG2["strategy_returns"]
    .cumsum()
    .apply(np.exp)
)
FAANG2["max_gross_performance"] =\
(
    FAANG2
    ["cumulative_returns"]
    .cummax()
)

# (
#     FAANG2
#     [["cumulative_returns", "max_gross_performance"]]
#     .dropna()
#     .plot(figsize = [18, 8]
#          )
# )
drawdown = FAANG2["max_gross_performance"] - FAANG2["cumulative_returns"]
drawdown.max()

## 最大回撤期

In [ ]:
ddlist = []
splist =[]
for i in range(1,len(FAANG2["max_gross_performance"])-1):
    if FAANG2["max_gross_performance"][i] == FAANG2["max_gross_performance"][i+1]:
        if FAANG2["max_gross_performance"][i] != FAANG2["max_gross_performance"][i-1]:
            ddlist.append(i)

for i in range(len(ddlist)-1):
    splist.append((FAANG2.index[ddlist[i+1]] - FAANG2.index[ddlist[i]]).days)

duration = max(splist)
duration

## 设置一个rolling期（以41天为例）以及一个上下波动的阈值（以4为例）
### 1.单纯画图

In [ ]:
FAANG5 =Daily_Closing_Price["WMT"].copy()
FAANG5 = pd.DataFrame(FAANG5)

#更改天数在这里
FAANG5["SMA_41"] =\
(
    FAANG5["WMT"]
    .rolling(window = 41)
    .mean()
)

#更改阈值在这里
threshold = 4

FAANG5["distance"] = FAANG5["WMT"] - FAANG5["SMA_41"]

(FAANG5["distance"]
 .dropna()
 .plot(figsize = [16, 6]
      )
)

plt.axhline(threshold, color = "blue", ls = "--")

plt.axhline(0, color = "green", ls = "--")

plt.axhline(-threshold, color = "red", ls = "--")


### 2.画出买入卖出点

In [ ]:
FAANG6 =FAANG5.copy()
FAANG6=FAANG6.reset_index()
(
FAANG5["distance"]
.dropna()
.plot(figsize = [16, 7],
      color = "grey",
      alpha = 0.80)
)

# Upper-bound threshold

plt.axhline(4,
            color = "blue",
            ls = "--")

# Lower-bound threshold

plt.axhline(-4,
            color = "blue",
            ls = "--")

# Sell Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] > 4,
           -1, np.nan)
)

# Buy Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] < -4,
           1, FAANG5["positions"])
)

# Market-Neutral Signal

FAANG5["positions"] =\
(
    np
    .where(FAANG5["distance"] * FAANG5["distance"].shift(1) < 0,
           0, FAANG5["positions"])
)
            
sum_positions = 0

for i in range(1994):
    if FAANG5["positions"][i] == 1:
        plt.scatter(FAANG6['Date'][i],1,color='red',marker='.',s=100)
        sum_positions  +=1
    elif FAANG5["positions"][i] == -1:
        plt.scatter(FAANG6['Date'][i],-1,color='blue',marker='.',s=100)
        sum_positions  +=1
    i +=1

### 3.对比strategy return和原股票return（我认为大部分算return都可以按这个来）

In [ ]:
FAANG5["RETURNS"] =\
(
    np
    .log(FAANG5["WMT"]
         /
         FAANG5["WMT"].shift(1)
        )
)
FAANG5["positions"] =\
    (FAANG5["positions"]
     .ffill()
    )
FAANG5["STRATEGY"] =\
    (
    FAANG5
    ["positions"]
    .shift(1)
    *
    FAANG5["RETURNS"]
    )
(
    FAANG5
    [["RETURNS", "STRATEGY"]]
    .dropna()
    .cumsum()
    .apply(np.exp)
    .plot(figsize = [18, 9]
         )
)

## sharpe ratio

In [ ]:
Portfolio["returns"] = Portfolio["total"].pct_change()

Sharpe =np.sqrt(253) * (Portfolio["returns"].mean() / Portfolio["returns"].std())

## Compound Annual Growth Rate，CAGR

In [ ]:
#计算交易日天数
days_I =\
(    (FAANG2.index[-1] - FAANG2.index[0])
     .days
)

FAANG2 =\
(
    FAANG2
    .dropna()
)

CAGR_I =\
(
    (
      (
        (FAANG2["strategy_returns"][-1]+1) / (FAANG2["strategy_returns"][0]+1)
      )
        **(365.0/days_I) 
    ) 
    - 1
)

CAGR_I 

## 抓取股票后由一个rolling（42天） 以及一个阈值（这里是两倍std）确定的上下限，制定策略
### 1.画图

In [ ]:
google["Log_return"] = np.log(google["Close"]/google["Close"].shift(1))

#更改rolling天数在这一段！一段！一段！
google["SMA_42"] = google["Close"].rolling(window=42).mean()
google["Distance"] = google["Close"] - google["SMA_42"]
google["Upper_threshold"] = 2*google["Close"].rolling(window=42).std()
google["Lower_threshold"] = -2*google["Close"].rolling(window=42).std()

google["Trading_position"] = np.where(google["Distance"] > google["Upper_threshold"], -1, np.nan)
google["Trading_position"] = np.where(google["Distance"] < google["Lower_threshold"], 1, google["Trading_position"])
google["Trading_position"] = np.where(google["Distance"]*google["Distance"].shift(1) < 0, 0, google["Trading_position"])
google["Trading_position"].ffill(inplace=True)

google1 = google.copy()
mask = (google1["Trading_position"]!=google1["Trading_position"].shift(1))
google1["Signal"] = google1["Trading_position"].where(mask, 0)

plt.figure(figsize=(18,9))
plt.plot(google1["Close"], label="Close")
plt.plot(google1["SMA_42"], label="SMA_42")
plt.plot(google1["SMA_42"]+google1["Upper_threshold"], label="Upper_threshold")
plt.plot(google1["SMA_42"]+google1["Lower_threshold"], label="Lower_threshold")
plt.plot(google1[google1["Signal"]==1]["Close"],"g^")
plt.plot(google1[google1["Signal"]==-1]["Close"],"rv")
plt.legend()
plt.show()

### 2.计算Sharpe ratio

In [ ]:
google1["Log_strategy"] = google1["Log_return"]*google1["Trading_position"].shift(1)
google1["Strategy"] = np.exp(google1["Log_strategy"])-1
sharpe_ratio = np.sqrt(253)*np.mean(google1["Strategy"])/np.std(google1["Strategy"])
sharpe_ratio

## 用之前的一定天数（以 3 天为例）预测明天的log return

In [ ]:
ge = pdr.get_data_yahoo("GE",dt.datetime(2011, 8, 1), dt.datetime(2020, 8, 1))
ge["Log_return"] = np.log(ge["Close"]/ge["Close"].shift(1))

#更改天数在这里
lags = 3
cols = []
for lag in range(1,lags+1):
    col = f"lag_{lag}"
    cols.append(col)
    ge[col] = ge["Log_return"].shift(lag)
ge1 = ge.copy()
ge1.dropna(inplace=True)
OLS = np.linalg.lstsq(ge1[cols],ge1["Log_return"])[0]
ge1["Prediction"] = np.dot(ge1[cols], OLS)

accuracy = np.sign(ge1["Prediction"]*ge1["Log_return"]).value_counts()
accuracy.values[0]/sum(accuracy)

#画图
# plt.figure(figsize=(18,9))
# plt.plot(ge1["Prediction"].cumsum(), label="Prediction_1_2_3")
# plt.legend()
# plt.show()

#计算Sharpe ratio
#np.sqrt(253)*np.mean((np.exp(ge1["Prediction"])-1))/np.std((np.exp(ge1["Prediction"])-1))

## MACD策略，长期(26天)短期(12天)差值与MACD signal(9天)

In [ ]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2017, 12, 31)
AAPL =\
(    web
     .get_data_yahoo("AAPL", 
                     start, 
                     end)
)
AAPL =AAPL.rename(columns={'Adj Close': 'Adj_Close'})
AAPL["mean_12"] =\
    (
    AAPL[["Adj_Close"]]
    .rolling(12)
    .mean()
    )
AAPL["mean_26"] =\
    (
    AAPL[["Adj_Close"]]
    .rolling(26)
    .mean()
    )
AAPL["MACD"]=AAPL["mean_12"]-AAPL["mean_26"]
AAPL["MACD_signal"] =\
    (
    AAPL[["MACD"]]
    .rolling(9)
    .mean()
    )
AAPL["Trading_Positions"] =\
    (
    np
    .where(AAPL["MACD"] > AAPL["MACD_signal"],
           1, -1)
    )

AAPL["Positions"] =\
(    AAPL["Trading_Positions"]
     .diff()
)

AAPL = AAPL.reset_index()
AAPL.set_index('Date', inplace=True)

fig =\
    (plt
     .figure(figsize = [16, 10]
            )
    )

sub =\
(    fig
    .add_subplot(111,
                 ylabel = "Stock Price")
)

(AAPL["Adj_Close"]
 .plot(ax = sub,
       color = "grey",
       lw = 0.80) # This is for closing price
)

(AAPL[["MACD_signal", 
              "MACD"]]
            .plot(ax = sub,
                  style = ["--", "--"],
                  lw = 0.80
                 )
)


# Buy

(
    sub
    .plot(AAPL.loc[AAPL['Positions'] == 2].index,#.astype(str),
          AAPL.Adj_Close[AAPL['Positions'] == 2],
          "g^",
          color = "green",
          markersize = 12)

)

# Sell

(
    sub
    .plot(AAPL.loc[AAPL['Positions'] == -2].index,#,
          AAPL.Adj_Close[AAPL['Positions'] == -2],
          "rv",
          color = "red",
          markersize = 12)
)

## find the `five` worst drawdown periods over the investment horizon and provide their net drawdown in % and duration, respectively.

In [ ]:
Portfolio['cumulative_returns'] =\
    Portfolio["cumulative_capital"]/1e5
Portfolio["max_gross_performance"] =\
(
    Portfolio
    ["cumulative_returns"]
    .cummax()
)

Portfolio['drawdown'] =\
    (Portfolio["max_gross_performance"]-Portfolio["cumulative_returns"])

Drawdown = pd.DataFrame(columns = ['start','end','max','duration'],
                        index = range(len(Portfolio[Portfolio['drawdown']==0])-1))

row=0
for i in range(len(Portfolio)) :
        for j in range(i+1,len(Portfolio)) :
            max_array = np.array([])
            if Portfolio['drawdown'][i] == 0 and Portfolio['drawdown'][j] == 0 :
                Drawdown['start'][row] = Portfolio.index[i]
                Drawdown['end'][row] = Portfolio.index[j]
                Drawdown['duration'][row] = (Drawdown['end'][row] - Drawdown['start'][row]).days
                for k in Portfolio['drawdown'].iloc[i:j+1] :
                    max_array = np.append(max_array , k)
                    Drawdown['max'][row] = max_array.max()*100
                row += 1
                break

for i in Drawdown['max'].astype('float64').nlargest(5).index :
    print(f"net drawdown = {Drawdown['max'][i]} , duration ={Drawdown['duration'][i]}days")